<div style="width:100%;">
    <img style="width:100%;" src="https://storage.googleapis.com/kaggle-datasets-images/1168771/1958110/65e4773b360f3e25b8331df57cfd4607/dataset-cover.jpg">
</div>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras import metrics
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

<h1 id="dataset" style="color:#b2b2b2; background:#fafafa; border-style:dotted;"> 
    <center>Dataset
        <a class="anchor-link" href="#dataset" target="_self">¶</a>
    </center>
</h1>

In [ ]:
path = '../input/heart-attack-analysis-prediction-dataset/heart.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
# split into features and labels
features = df.drop(columns=['output'])
labels = df['output'].values

# normalize features
scaler = MinMaxScaler()
scaler.fit(features)
features = scaler.transform(features)

# shuffle features and labels
features, labels = shuffle(features, labels)

# labels encoder
y_class = LabelEncoder().fit_transform(labels)
n_class = len(np.unique(y_class))
n_features = features.shape[1]

# split ddata
X_train, X_test, y_train, y_test, y_train_class, y_test_class = train_test_split(features, labels, y_class, test_size=0.33, random_state=1)

<h1 id="regression" style="color:#b2b2b2; background:#fafafa; border-style:dotted;"> 
    <center>Linear Regression Metric
        <a class="anchor-link" href="#regression" target="_self">¶</a>
    </center>
</h1>

In [ ]:
def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

<h1 id="model" style="color:#b2b2b2; background:#fafafa; border-style:dotted;"> 
    <center>Regression & Classification Model
        <a class="anchor-link" href="#model" target="_self">¶</a>
    </center>
</h1>

In [ ]:
visible = Input(shape=(n_features,))

# base model
hidden1 = Dense(32, activation='relu', kernel_initializer='he_normal')(visible)
hidden2 = Dense(16, activation='relu', kernel_initializer='he_normal')(hidden1)

# regression output
out_reg = Dense(1, activation='linear')(hidden2)

# classification output
out_clas = Dense(n_class, activation='softmax')(hidden2)

# define model
model = Model(inputs=visible, outputs=[out_reg, out_clas])

# compile the keras model
model.compile(loss=['mse','sparse_categorical_crossentropy'], 
              metrics=[soft_acc, metrics.SparseCategoricalAccuracy()], optimizer='adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, [y_train,y_train_class], epochs=20, batch_size=32, verbose=0, validation_data=(X_test, [y_test,y_test_class]))

<h1 id="analysis" style="color:#b2b2b2; background:#fafafa; border-style:dotted;"> 
    <center>Analysis
        <a class="anchor-link" href="#analysis" target="_self">¶</a>
    </center>
</h1>

In [ ]:
all_keys = list(history.history.keys())[8:]
loss_keys = all_keys[:2]
acc_keys = all_keys[2:]

## Regression

In [ ]:
losses = history.history[loss_keys[0]]
plt.figure(figsize=(14,8))
plt.title('Regression Losses')
plt.plot(np.arange(len(losses)), losses)

In [ ]:
accuracies = history.history[acc_keys[0]]
plt.figure(figsize=(14,8))
plt.title('Regression Accuracy')
plt.plot(np.arange(len(accuracies)), accuracies)

## Classification

In [ ]:
losses = history.history[loss_keys[1]]
plt.figure(figsize=(14,8))
plt.title('Classification Losses')
plt.plot(np.arange(len(losses)), losses)

In [ ]:
accuracies = history.history[acc_keys[-1]]
plt.figure(figsize=(14,8))
plt.title('Classification Accuracy')
plt.plot(np.arange(len(accuracies)), accuracies)